In [1]:
# !pip install -q pytorch-lightning
!pip install accelerate -U
!pip install -q --upgrade transformers datasets rouge_score
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [2]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [3]:
# transformers.__version__

In [4]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
# import pytorch_lightning as pl
import torch
# from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse
import datetime

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from IPython.display import display, HTML
import random
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset, load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

nltk.download('punkt')

import wandb
wandb.login(key='242e3440871263aadfecec7d77eb859714ca3269')

2024-06-07 23:27:42.925869: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 23:27:42.926009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 23:27:43.086227: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#### References: [Official](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/summarization.ipynb#scrollTo=vc0BSBLIIrJQ), [Community](https://colab.research.google.com/github/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb#scrollTo=6R9d7ELIpX9F)

In [5]:
BASE_PATH = ""

MODEL_PATH = "/kaggle/input/claimdecomp3/cast_flant5_large_strategyq_with_answermodel"
TOKENIZER_PATH = "google/flan-t5-large"
SAVE_PATH = BASE_PATH + "cast_flant5_large_strategyq_with_answer/"
LOGGING_PATH = BASE_PATH + "cast_flant5_large_strategyq_with_answer_logs/"
SAVE_MODEL_PATH = BASE_PATH + "cast_flant5_large_strategyq_with_answer:model/"

In [6]:
# !pip install jsonlines
# import jsonlines


In [7]:
# def read_jsonlines(file_path):
#     data = []
#     with jsonlines.open(file_path) as reader:
#         for obj in reader:
#             claim = obj['claim']
#             # Assuming you want to use the 'statements-negate' field for decomposition
#             decomposition = ' '.join(obj['annotations'][0]['questions'])
#             data.append({'claim': claim, 'decomposition': decomposition})
#     return data

# data_subq = read_jsonlines("/kaggle/input/claimdecomp3/train.jsonl")
# val = read_jsonlines("/kaggle/input/claimdecomp3/dev.jsonl")


# print(data_subq[0])

In [8]:
# from sklearn.model_selection import train_test_split
# train,val = train_test_split(data_subq,test_size=0.10,random_state=42)


In [9]:
# train = pd.DataFrame(train)


# val = pd.DataFrame(val)

In [10]:
# train.to_csv("train_strategyq_with_answer.csv",index=False)
# val.to_csv("val_strategyq_with_answer.csv",index=False)

# Loading and processing the data

In [11]:
# TRAIN_FILE_PATH = "train_strategyq_with_answer.csv"
# VAL_FILE_PATH = "val_strategyq_with_answer.csv"

In [12]:
# data_files = {}
# data_files["train"] = TRAIN_FILE_PATH
# data_files["validation"] = VAL_FILE_PATH

In [13]:
# import json
# with open(TRAIN_FILE_PATH) as f:
#   val_data = json.load(f)

In [14]:
# xmetric = load_metric("rouge")
# data_files
# raw_datasets = load_dataset("csv", data_files=data_files)


In [15]:
metric = load_metric("rouge")


/tmp/ipykernel_18/858454605.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [16]:
# raw_datasets["train"][0]

In [17]:
# def show_random_elements(dataset, num_examples=5):
#     assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
#     picks = []
#     for _ in range(num_examples):
#         pick = random.randint(0, len(dataset)-1)
#         while pick in picks:
#             pick = random.randint(0, len(dataset)-1)
#         picks.append(pick)

#     df = pd.DataFrame(dataset[picks])
#     for column, typ in dataset.features.items():
#         if isinstance(typ, datasets.ClassLabel):
#             df[column] = df[column].transform(lambda i: typ.names[i])
#     display(HTML(df.to_html()))

In [18]:
# show_random_elements(raw_datasets["train"])

## Tokenization and Dataset Preparation

In [19]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [20]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-large', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>

In [21]:
max_input_length = 64
max_target_length = 128

In [22]:

def preprocess_function(examples, prefix="decompose the compositional question:"):
    inputs = [prefix + doc for doc in examples["claim"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # The "labels" are the tokenized outputs:
    print(examples["decomposition"])
    labels = tokenizer(text_target=examples["decomposition"], max_length=max_target_length, truncation=True)
    print(labels)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
# tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

In [24]:
# tokenized_datasets

# Building the model

## Metrics

In [25]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     print(predictions[0])
#     predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     print("decoded gen",decoded_preds)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Rouge expects a newline after each sentence
#     decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     # Extract a few results
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

#     # Add mean generated length
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)

#     return {k: round(v, 4) for k, v in result.items()}

## Loading the model

In [26]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

In [27]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

## Training Args

In [28]:
# epochs = 6
# batch_size = 8
# lr = 2e-5

In [29]:
# args = Seq2SeqTrainingArguments(
#     output_dir=SAVE_PATH,
#     learning_rate=lr,
#     do_train = True,
#     do_eval = True,
#     evaluation_strategy="steps",
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     load_best_model_at_end=True,
#     num_train_epochs=epochs,
#     predict_with_generate=True,
#     generation_max_length=512,
#     logging_dir=LOGGING_PATH,
#     logging_steps=300,
#     save_steps=300,
#     report_to = "wandb",
# )

In [30]:
# wandb_run = wandb.init(
#     project="flant5_subq",
#     config={
#         "per_device_train_batch_size": batch_size,
#         "learning_rate": lr})

# now = datetime.datetime.now()
# current_time = now.strftime("%m/%d/%Y, %H:%M:%S")
# wandb_run.name = "run_" + current_time

In [31]:
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [32]:
# args = Seq2SeqTrainingArguments(
#     output_dir=SAVE_PATH,
#     learning_rate=lr,
#     do_train = True,
#     do_eval = True,
#     evaluation_strategy="steps",
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     load_best_model_at_end=True,
#     num_train_epochs=epochs,
#     predict_with_generate=True,
#     generation_max_length=512,
#     logging_dir=LOGGING_PATH,
#     logging_steps=300,
#     save_steps=300,
#     report_to = "wandb",
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=args,
#     data_collator=data_collator,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics)

In [33]:
# torch.cuda.empty_cache()

# Training

In [34]:
# trainer.evaluate() # Evaluation before fine-tuning

In [35]:
# val

In [36]:
# # %%wandb

# trainer.train()

In [37]:
# trainer.evaluate()

In [38]:
# trainer.save_model(SAVE_MODEL_PATH)

In [39]:
# wandb_run.finish()

In [40]:
# model = AutoModelForSeq2SeqLM.from_pretrained(SAVE_MODEL_PATH)

In [41]:
import json
with open("/kaggle/input/claimdecomp3/val_claims_quantemp.json") as f:
  train_data = json.load(f)
len(train_data
    )
train_data = train_data[2000:]

In [42]:
claims = [claim['claim'] for claim in train_data]
claims[0]

'In 2021, CNN reported that Coca-Cola had announced it was "changing Santa Claus from a caucasian man to an afro-american man."'

# Evaluation

In [43]:
def generate_output(test_samples, model):
    inputs = tokenizer(
        test_samples,
        max_length=128,
        return_tensors="pt")

    input_ids = inputs.input_ids.to(model.device)
    print(input_ids)
    print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask,min_length = 64, max_length = 128, do_sample=True, top_p=0.95, top_k=50)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return output_str

In [44]:
ans = []
for i in range(len(claims)):
    print(i)
    ans.append(generate_output("decompose the compositional question:"+claims[i], model))
    
    


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0
tensor([[   20,   287,  2748,    15,     8,  5761,   138,   822,    10,  1570,
           460,  2658,     6, 19602,  2196,    24, 25417,    18,  9939,     9,
           141,  2162,    34,    47,    96, 12757,  4625, 22021,    45,     3,
             9,   212, 18283, 10488,   388,    12,    46,     3,     9,  6155,
            18, 23064,    29,   388,   535,     1]])
decompose the compositional question:In 2021, CNN reported that Coca-Cola had announced it was "changing Santa Claus from a caucasian man to an afro-american man."
1
tensor([[   20,   287,  2748,    15,     8,  5761,   138,   822,    10,  3612,
             7,    17,   215,     6, 11882,   751,  2111,  5743,    13,     8,
          2902,     6,   780,    79,  1520,   163,   220,  5953,    13,     8,
          6116,    16,     8,   538,  1384,    11, 10738,    13,     8,  6116,
            16,     8,   538,  7819,     5,     1]])
decompose the compositional question:Last year, Democrats won nearly 50% of the vote, yet they

In [45]:
# print(len(claims))
# print(ans)
# print(claims[0])

In [46]:
import csv
with open('Claim_decomp_val_2000_second', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(ans)


In [47]:
# ans

In [48]:
# idx = 4
# part = 'validation'

# sample = raw_datasets[part][idx]
# sample_ = tokenized_datasets[part][idx]

In [49]:
# # Checking if the tokenizer is working correctly

# input_ids, labels = sample_['input_ids'], sample_['labels']
# print(tokenizer.decode(input_ids, skip_special_tokens=True), tokenizer.decode(labels, skip_special_tokens=True), sep="\n")

In [50]:
# !cp -r  /content/cast_flant5_large_strategyq_with_answer:model /content/drive/MyDrive/CSAT

In [51]:
# print("decompose the compositional question:"+sample['claim'], sample['decomposition'], sep="\n", end="\n\n")
# generate_output("decompose the compositional question:"+sample['claim'], model)

In [52]:
# # import pandas as pd
# # test_data = pd.read_csv("test_vw.csv")
# def read_jsonlines2(file_path):
#     data = []
#     with jsonlines.open(file_path) as reader:
#         for obj in reader:
#             claim = obj['claim']
#             # Assuming you want to use the 'statements-negate' field for decomposition
#             decomposition = ' '.join(obj['annotations'][0]['statements'])
#             data.append({'claim': claim, 'decomposition': decomposition})
#     return data

# test_data = read_jsonlines2("/kaggle/input/claimdecomp2/test.jsonl")

# print(test_data[0])
# test_data = pd.DataFrame(test_data)

In [53]:
# test_data

In [54]:
# seq_list = []
# for query_paa in test_data["claim"].values:
#     seq_list.append(generate_output(query_paa,model))
# len(seq_list)


In [55]:
# seq_list

In [56]:
# machine_generated = [seq[0] for seq in seq_list]
# machine_generated

In [57]:
# test_data = pd.read_csv("test_vw.csv")
# queries_list = test_data["title"].values.tolist()

In [58]:
# final_test_df = {"queries":queries_list,"gpt_out":machine_generated, "ground_truth":list(test_data["ground_truth"].values),"description":list(test_data["description"].values),"precision":[],"recall":[],"f1":[]}

In [59]:
# final_test_df = {"gpt_out":machine_generated, "ground_truth":list(test_data["decomposition"].values),"description":list(test_data["claim"].values),"precision":[],"recall":[],"f1":[]}

In [60]:
# actual = []
# predicted = []
# for target in list(test_data["decomposition"].values):
#   actual.append([target.split(" ")])

# for pred in machine_generated:
#   predicted.append(pred.split(" "))



In [61]:
# from nltk.translate.bleu_score import corpus_bleu

# bleu_dic = {}
# bleu_dic['1-grams'] = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
# bleu_dic['1-2-grams'] = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
# bleu_dic['1-3-grams'] = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
# bleu_dic['1-4-grams'] = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))

In [62]:
# bleu_dic['1-4-grams']

In [63]:
# bleu_dic['1-3-grams']

In [64]:
# bleu_dic['1-2-grams']

In [65]:
# !pip install datasets
# !pip install bert_score

In [66]:
# from datasets import load_metric
# bertscore_metric = load_metric('bertscore')


In [67]:
# bert_scores = bertscore_metric.compute(predictions=machine_generated, references=list(test_data["decomposition"].values),model_type="distilbert-base-uncased", lang="en")
# print(bert_scores)
# # Normally, we use the f1-score attribute
# print(bert_scores['f1'])

In [68]:
# final_test_df["precision"] = bert_scores["precision"]
# final_test_df["recall"] = bert_scores["recall"]
# final_test_df["f1"] = bert_scores["f1"]

In [69]:
# final_test_df = pd.DataFrame(final_test_df)
# final_test_df.to_csv("bart_out_job_desc_gen.csv",index=False)
# print(final_test_df)

In [70]:
# from statistics import mean,median
# print("f1",mean(bert_scores['f1']))
# print("recall:",mean(bert_scores['recall']))
# print("precision:",mean(bert_scores['precision']))

In [71]:
# !zip -r bart_job_desc_gen_model.zip /bart_job_desc_gen_model

In [72]:
# !zip -r bart_reconstruction.zip /content/bart_reconstruction